In [ ]:
import numpy as np
import xarray as xr
import pandas as pd
import seaborn as sns
import mplotutils as mpu
from netCDF4 import Dataset
import metpy.calc as mpcalc
from metpy.units import units
from datetime import datetime
import matplotlib.pyplot as plt
from wrf import getvar, ll_to_xy, to_np, Constants
from metpy.plots import Hodograph, SkewT
from matplotlib.colors import ListedColormap
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

%matplotlib inline

# Skew T - log p Diagram

In [ ]:
def sounding(save_name, title=True, save=False):
    # Predefine the dates and time ranges for the different case studies
    if save_name == 'case_study_1':
        start_date = '2018-05-22 16:30'
        end_date = '2018-05-22 17:00'
    elif save_name == 'case_study_2':
        start_date = '2018-05-09 13:05'
        end_date = '2018-05-09 14:05'
    elif save_name == 'case_study_3':
        start_date = '2018-05-12 19:45'
        end_date = '2018-05-12 20:15'
    elif save_name == 'case_study_4':
        start_date = '2018-05-10 00:30'
        end_date = '2018-05-10 01:00'
    elif save_name == 'case_study_5':
        start_date = '2018-05-30 14:20'
        end_date = '2018-05-30 14:50'    

    # Creating a daterange
    date_range = pd.date_range(start=start_date, end=end_date, freq='5min')

    # Converting it into datetime objects
    date_range2 = [datetime.strptime(str(d), '%Y-%m-%d %H:%M:%S') for d in date_range]
    
    for date in date_range2:
        ### Predefine some variables ###
        tile = False
        save = True

        if save_name == 'case_study_1':
            lat = 47.4961
            lon = 7.56971   
        elif save_name == 'case_study_2':
            lat = 47.0546
            lon = 7.63891
        elif save_name == 'case_study_3':
            lat = 47.5522
            lon = 8.64449
        elif save_name == 'case_study_4':
            lat = 47.629
            lon = 8.67235
        elif save_name == 'case_study_5':
            lat = 46.4482
            lon = 6.95974        

        ### Plotting procedure ###    
        filename = '/scratch3/thomasl/work/data/{}/wrfout_d02_{}_{}'.format(save_name, str(date)[0:10], 
                                                                                             str(date)[11:])


        # Get computed variables by wrf-python
        ncfile = Dataset(filename)

        pressure = getvar(ncfile, 'pressure')
        height = getvar(ncfile, 'height')
        temperature = getvar(ncfile, 'tc')
        dewpoint = getvar(ncfile, 'td')
        wspd = getvar(ncfile, 'wspd_wdir')[0,:]
        wdir = getvar(ncfile, 'wspd_wdir')[1,:]

        # Add computed variables to dataset
        ds = xr.open_dataset(filename)

        ds['pressure'] = (['bottom_top', 'south_north', 'west_east'], pressure)
        ds['height'] = (['bottom_top', 'south_north', 'west_east'], height)
        ds['temperature'] = (['bottom_top', 'south_north', 'west_east'], temperature)
        ds['dewpoint'] = (['bottom_top', 'south_north', 'west_east'], dewpoint)
        ds['wspd'] = (['bottom_top', 'south_north', 'west_east'], wspd)
        ds['wdir'] = (['bottom_top', 'south_north', 'west_east'], wdir)

        # Select location of sounding
        # Make a function that deals with points which have longitudes > STAND_LON
        def rotated_ll_to_xy(wrfFile, latitude, longitude):
            res = ll_to_xy(wrfFile, latitude=latitude, longitude=longitude)
            diffPix = np.round((2*Constants.WRF_EARTH_RADIUS*Constants.PI)/wrfFile.DX,0)
            xs = np.where(np.ravel(longitude) >= wrfFile.STAND_LON*-1, res[0]+diffPix, res[0])
            res[0] = xs[0]
            return(res)

        x_y = rotated_ll_to_xy(ncfile, lat, lon)

        ds_point = ds.sel(west_east=x_y[0], south_north=x_y[1])

        # Get variables for location
        p = ds_point['pressure'].values * units.hPa
        T = ds_point['temperature'].values * units.degC
        Td = ds_point['dewpoint'].values * units.degC
        wind_speed = ds_point['wspd'].values * units.knots
        wind_dir = ds_point['wdir'].values * units.degrees

        # Limit pressure level for wind barbs
        p_200 = ds_point['pressure']
        p_200 = p_200.where(p_200 > 200)
        p_200 = p_200.values * units.hPa

        # Calculate wind components
        u, v = mpcalc.wind_components(wind_speed, wind_dir)

        # Calculate the LCL
        lcl_pressure, lcl_temperature = mpcalc.lcl(p[0], T[0], Td[0])

        # Calculate the parcel profile
        parcel_prof = mpcalc.parcel_profile(p, T[0], Td[0]).to('degC')

        # Create figure
        fig = plt.figure(figsize=(10, 10))
        skew = SkewT(fig, rotation=45)

        # Plot the data
        skew.plot(p, T, 'r', linewidth=2)
        skew.plot(p, Td, 'g', linewidth=2)
        skew.plot_barbs(p_200, u, v)
        skew.ax.set_ylim(1000, 100)
        skew.ax.set_xlim(-40, 60)

        # Plot LCL as black dot
        skew.plot(lcl_pressure, lcl_temperature, 'ko', markerfacecolor='black')

        # Plot the parcel profile as a black line
        skew.plot(p, parcel_prof, 'k', linewidth=2)

        # Shade areas of CAPE and CIN
        skew.shade_cin(p, T, parcel_prof)
        skew.shade_cape(p, T, parcel_prof)

        # Plot a zero degree isotherm
        skew.ax.axvline(0, color='c', linestyle='--', linewidth=2)

        # Add the relevant special lines
        skew.plot_dry_adiabats()
        skew.plot_moist_adiabats()
        skew.plot_mixing_lines()

        # Create a hodograph
        ax_hod = inset_axes(skew.ax, '30%', '30%', loc=1)
        h = Hodograph(ax_hod, component_range=30.)
        h.add_grid(increment=20)

        # Add wind speed colored line 
        cmap = ListedColormap(sns.cubehelix_palette(10, start=.5, rot=-.75))
        if save_name == 'case_study_2' or save_name == 'case_study_4':
            levels = np.linspace(0, 50, 11)
        else:
            levels = np.linspace(0, 20, 11)
        wind_speed_h = h.plot_colormapped(u, v, wind_speed, intervals=levels, cmap=cmap)

        # Add color bar for hodograph
        cbar_h = mpu.colorbar(wind_speed_h, ax_hod, orientation='vertical', 
                              aspect=30, shrink=.05, pad=0.1, extend='max')
        cbar_h.set_label('Wind Speed [$kn$]', fontsize=17.5)
        cbar_h.set_ticks(levels)
        cbar_h.ax.tick_params(labelsize=15)

        # Set label of axes
        skew.ax.set_ylabel('Pressure [$hPa$]', fontsize=20)
        skew.ax.set_xlabel('Temperature [$°C$]', fontsize=20)
        skew.ax.tick_params(axis='both', which='major', labelsize=17.5, length=8, width=1.5)
        skew.ax.tick_params(axis='both', which='minor', length=4, width=1)

        # Make nicetime
        nicetime = pd.to_datetime(date)

        # Add title
        if title == True:
            skew.ax.set_title('')
            skew.ax.set_title('Valid time: {} UTC'.format(nicetime.strftime('%Y-%m-%d %H:%M')), 
                              loc='right', fontsize=12.5);
            skew.ax.set_title(r'$\mathbf{Skew\ T-log\ p\ Diagram}$' + '\n @ Initiation Location ({}, {})'.format(lat, lon),
                              loc='left', fontsize=15)

        plt.show()

        # Save figure
        if save == True:
            fig.savefig('/scratch3/thomasl/work/retrospective_part/' \
                        '{}/soundings/sounding_{}.png'.format(save_name, date), 
                        bbox_inches='tight', dpi=300)

In [ ]:
sounding(save_name='case_study_5', title=False, save=True)